In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
random.seed(42)
np.random.seed(42)

In [3]:
#with open("../ressources/llm_evaluated_s14s15s16_n300_24_12_20_target.pickle", "rb") as file:
with open("../scripts/debug.pickle", "rb") as file:
#with open("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle", "rb") as file:
#with open("../ressources/llm_evaluated_concat_s14n600_s15n600_train_2024-12-12_16.pickle", "rb") as file:
#with open("testdata.pkl", "rb") as file:
    original_df = pickle.load(file)

In [4]:
original_df

,description,configuration,parameters,argos,type,scores_bt1,scores_bt2,bt1,bt2,scores_bt1_scaled,scores_bt2_scaled
0,The environment features a circle composed of ...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-2.643918,-9.895358,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,None,None
1,The rectangular area has dimensions 4.49 m x 7...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-7.519995,-7.693951,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,None,None
2,"With a radius of 3.54 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-29.58582,-28.77883,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,None,None
3,"In this setting, a circular arena with a radiu...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-53.61011,-12.167697,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,None,None
4,The environment is constructed as a rectangula...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-21.09988,-21.27474,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,None,None


In [5]:
#mod = original_df.drop("index",axis=1)

In [6]:
#mod.to_pickle("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle")

In [7]:
df = original_df

In [8]:
def rescale_score(score, df, category):
    df_cat = df[df.type == category]
    min_score = min(df_cat.scores_bt1.min(), df_cat.scores_bt2.min())
    max_score = max(df_cat.scores_bt1.max(), df_cat.scores_bt2.max())
    print(score,min_score, max_score,float(score-min_score),float(max_score - min_score))
    score_scaled = float(score-min_score)/float(max_score - min_score)
    print(score_scaled)
    
    return score_scaled
df["scores_bt1_scaled"] = df.apply(lambda row: rescale_score(row["scores_bt1"], df, row["type"]), axis=1)
df["scores_bt2_scaled"] = df.apply(lambda row: rescale_score(row["scores_bt2"], df, row["type"]), axis=1)
df


-2.6439179999999998 -53.61011 -2.6439179999999998 50.966192 50.966192
1.0
-7.519995 -29.585820000000002 -7.519995 22.065825000000004 22.065825000000004
1.0
-29.585820000000002 -29.585820000000002 -7.519995 0.0 22.065825000000004
0.0
-53.61011 -53.61011 -2.6439179999999998 0.0 50.966192
0.0
-21.09988 -29.585820000000002 -7.519995 8.485940000000003 22.065825000000004
0.3845738829162291
-9.895358 -53.61011 -2.6439179999999998 43.714752 50.966192
0.8577205846573744
-7.693951 -29.585820000000002 -7.519995 21.891869 22.065825000000004
0.9921164968905535
-28.77883 -29.585820000000002 -7.519995 0.8069900000000025 22.065825000000004
0.036571938733312825
-12.167697 -53.61011 -2.6439179999999998 41.442413 50.966192
0.8131353623594245
-21.27474 -29.585820000000002 -7.519995 8.31108 22.065825000000004
0.3766494114767972


,description,configuration,parameters,argos,type,scores_bt1,scores_bt2,bt1,bt2,scores_bt1_scaled,scores_bt2_scaled
0,The environment features a circle composed of ...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-2.643918,-9.895358,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,1.000000,0.857721
1,The rectangular area has dimensions 4.49 m x 7...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-7.519995,-7.693951,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,1.000000,0.992116
2,"With a radius of 3.54 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-29.58582,-28.77883,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,0.000000,0.036572
3,"In this setting, a circular arena with a radiu...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-53.61011,-12.167697,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,0.000000,0.813135
4,The environment is constructed as a rectangula...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-21.09988,-21.27474,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,0.384574,0.376649


In [9]:
df.shape

(5, 11)